In [1]:
# Basic Libraries
import pandas as pd
import modin.pandas as n_pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import random
import os

# Datatable Import
import datatable as dt

# Deep Learning Pytorch Library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchinfo import summary
torch.cuda.is_available()
torch.cuda.empty_cache()


In [2]:
# Import using datatable
X = dt.fread("X_MLP_DelayLine2_all_dims.csv")
Y = dt.fread("Y_MLP_DelayLine2_all_dims.csv")

# Convert to pandas
X = X.to_pandas()
X = X.dropna()
Y = Y.to_pandas()
Y = Y.dropna()

In [3]:
print(X.shape)
print(Y.shape)

(60296269, 34)
(60296269, 2)


In [4]:
# Define all dataset for train
all_dataX = Variable(torch.Tensor(X.values)) # .cuda()
all_dataY = Variable(torch.Tensor(Y.values)) # .cuda()

# Delete X and Y from memory
del X
del Y

In [5]:
# Create the Dataset
ds = TensorDataset(all_dataX, all_dataY)
del all_dataX, all_dataY

In [6]:
# Define the batch size
BatchSize = 10000

g = torch.Generator()
g.manual_seed(123)

# Create the DataLoader
dl = DataLoader(ds, batch_size=BatchSize, shuffle=False, num_workers= 8, pin_memory=True, drop_last=True, generator=g) #shuffle was at false

In [7]:
class Model(nn.Module):
    def __init__(self, in_features=6, h1=200, h2=100, h3 =25, out_features=2):
        super().__init__()
        
        # input layer -> 1 hidden -> 2 hidden -> output 
        self.fc1 = nn.Linear(in_features,h1) # input layer (+28 to include the dimensions)
        self.dropout = nn.Dropout(p = 0.1)
        self.fc2 = nn.Linear(h1 + 28, h2)            # hidden layer
        self.fc3 = nn.Linear(h2,h2)  # output layer
        self.fc4 = nn.Linear(h2,h3)
        self.fc5 = nn.Linear(h3, out_features)  # output layer
                
    def forward(self, x, dimensions):
        x = F.elu(self.fc1(x))
        in_fc = torch.cat((x.cuda(), dimensions.cuda()), dim=1).cuda()
        x = F.elu(self.fc2(in_fc))
        x = F.sigmoid(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = self.fc5(x)
        return x

In [8]:
torch.manual_seed(4)
model = Model()
model = torch.nn.DataParallel(model, device_ids=[0,1]).cuda()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #0.0001
summary(model)

Layer (type:depth-idx)                   Param #
DataParallel                             --
├─Model: 1-1                             --
│    └─Linear: 2-1                       1,400
│    └─Dropout: 2-2                      --
│    └─Linear: 2-3                       22,900
│    └─Linear: 2-4                       10,100
│    └─Linear: 2-5                       2,525
│    └─Linear: 2-6                       52
Total params: 36,977
Trainable params: 36,977
Non-trainable params: 0

In [9]:
# Define the number of epochs
number_of_epochs = 60

# Define the start time 
start_time = time.time()

# Variable to store the losses per epoch
losses = []
errors = []

# Instanciate that the model is in training mode
model.train()

# For every epoch defined...
for epoch in range(number_of_epochs):
    
    # Feed data in batches
    for batch_features, batch_labels in dl:
        
        # Extract input waves 
        input_waves = batch_features[:,0:6]
        input_waves = input_waves.cuda()

        # Extract the dimensions from all batch
        input_dimensions = batch_features[:,6:(batch_features.shape[1]+1)]
        input_dimension = input_dimensions.cuda()

        # Pass batch labels to GPU
        batch_labels = batch_labels.cuda()

        # Initialize the gradient to avoid value agregations
        optimizer.zero_grad()       
      
        # Predict the model using the training set
        y_pred = model(input_waves, input_dimensions)

        # Obtain the loss and the error
        loss = criterion(y_pred, batch_labels)
        error = y_pred - batch_labels
        
        # Backpropagation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1) # gradient cliping to avoid exploding gradients
        optimizer.step()

    # Append the losses to make the plot
    losses.append(loss.item())
    
    # Append the errors to obtain the metrics to create the noise
    errors.append(error)
    
    # Print the results
    if epoch % 2 == 0:
        print("-----> Epoch: %d, ----->loss: %.5E" % (epoch, loss.item()))
        print("Learning Rate: %.6E " % optimizer.param_groups[0]["lr"])
            
# Print the execution time
print(f'\nDuration: {(time.time() - start_time)/60:.0f} Min')

-----> Epoch: 0, ----->loss: 3.57529E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 2, ----->loss: 3.36445E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 4, ----->loss: 3.34465E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 6, ----->loss: 3.30595E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 8, ----->loss: 3.17372E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 10, ----->loss: 3.13362E-04
Learning Rate: 1.000000E-04 
-----> Epoch: 12, ----->loss: 3.10381E-04
Learning Rate: 1.000000E-04 


In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, '/home/aamaral/Desktop/BigDataset/Evaluation/MLP_DelayLine2_Sig_all_8.pt')